In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict

In [2]:
class Server:                                       # 服务器(cache)
    def __init__(self, space):
        self.space = space                          # cache大小
        self.remain = space                         # cache剩余空间
        self.cache = OrderedDict()                  # OrderDict() 模拟cache LRU方法
        self.hit_count = 0                          # 命中次数
        self.bite_hit_count = 0
        self.miss_count = 0                         # 未命中次数
        self.bite_miss_count = 0

    def _hit(self, fid, size):
        self.hit_count += 1
        self.bite_hit_count += size
        self.cache.move_to_end(fid)

    def _miss(self, fid, size):
        self.miss_count += 1
        self.bite_miss_count += size
        while self.remain < size:
            self.remain += self.cache.popitem(last=False)[-1]  # pop出第一个item
        self.cache[fid] = size
        self.remain -= size

    def handle(self, fid, size):                         # 处理一次请求
        if fid in self.cache.keys():
            self._hit(fid, size)
        else:
            self._miss(fid, size)

    def hit_rate(self):
        try:
            return self.hit_count / (self.hit_count + self.miss_count)
        except:
            return "Server has not been requested yet!"
    
    def bite_hit_rate(self):
        try:
            return self.bite_hit_count / (self.bite_hit_count + self.bite_miss_count)
        except:
            return "Server has not been requested yet!"

In [3]:
def merger(t1, t2):
    fid = []
    for i in range(len(t1)):
        fid.append(t1['fid'][i])
        fid.append(t2['fid'][i])
    return pd.DataFrame({'fid':fid, 'size': [1] * (2 * len(t1))})

In [26]:
def make_requests(df):
    for row in df.itertuples():
        yield getattr(row, 'fid'), getattr(row, 'size')

In [73]:
trace1 = pd.DataFrame({'fid': [1, 2, 3, 1, 2, 3, 1, 2, 3], 'size': [1, 1, 1, 1, 1, 1, 1, 1, 1]})
trace2 = pd.DataFrame({'fid': [5, 6, 7, 8, 5, 6, 7, 8], 'size': [1, 1, 1, 1, 1, 1, 1, 1]})
#trace_merge = merger(trace1, trace2)
trace_merge = pd.DataFrame({'fid': [1, 2, 5, 6, 7, 3, 8, 1, 2, 5, 6, 7, 3, 8, 1, 2, 3], 
                            'size': [1] * 17})

small_cache_1 = Server(3)
small_cache_2 = Server(3)
big_cache = Server(6)

for fid, size in make_requests(trace_merge):
    big_cache.handle(fid, size)
    
for fid, size in make_requests(trace1):
    small_cache_1.handle(fid, size)

for fid, size in make_requests(trace2):
    small_cache_2.handle(fid, size)

In [74]:
big_cache.hit_count

1

In [75]:
small_cache_1.hit_count

6

In [76]:
small_cache_2.hit_count

0